# Converting Scikit-learn Models to Tensorflow

We have trained a model in scikit previously. Now we replicate the same setup in tensorflow.


In [1]:
from tqdm.notebook import tqdm
import json
import re
import numpy as np 
import pandas as pd #
import tensorflow as tf
from tensorflow import keras
from keras.utils import pad_sequences
from keras import layers
from keras.preprocessing.text import Tokenizer
import tensorflow_constrained_optimization as tfco
import tensorflow_model_analysis as tfma
import fairness_indicators as fi
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from time import time
import re

# For kaggle only
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
tqdm.pandas()

if tf.__version__ < "2.0.0":
  tf.enable_eager_execution()
  print("Eager execution enabled.")
else:
  print("Eager execution enabled by default.")

print("TensorFlow " + tf.__version__)
print("FI " + fi.version.__version__)
print("TFMA " + tfma.VERSION_STRING)

Eager execution enabled by default.
TensorFlow 2.9.0
FI 0.44.0
TFMA 0.44.0


While the input is sentences, the classification is one of 4 labels. A model can only process numbers, therefore we need to convert these labels to ordinals.

For TS JS, we can piggyback on this pretrained model: https://github.com/tensorflow/tfjs-models/tree/master/universal-sentence-encoder. The Universal Sentence Encoder is a model that encodes text into 512-dimensional and uses an 8k word piece vocabulary.

We will do cleanup in python of the trained sentenses, and we'll try to find lemmatizers and stopword libraries in JS also.



In [2]:
# DATA = "/kaggle/input/linkedin/anonLinkedInProfiles.csv"
DATA = "./data/anonLinkedInProfiles.csv"
DATA_LEN = 1400
LABEL_DICT = dict()

data = pd.concat([chunk for chunk in tqdm(pd.read_csv(DATA, chunksize=1000), desc=f'Loadin {DATA}')])
print(f'Shape: {data.shape}, does it have NAs:\n{data.isna().any()}')

data = data.sample(DATA_LEN, random_state=200)
data = data.reset_index() # Reset index, since we will do operations on it!
print(f'Resampled Shape: {data.shape}')

def _get_or_set_label(x):
    if x not in LABEL_DICT:
        LABEL_DICT[x] = len(LABEL_DICT)
    return LABEL_DICT[x]

X = data['titles'].astype(str) +  ' ' + data['descriptions'].astype(str)
Y = data['class'].apply(lambda x: _get_or_set_label(x)).astype(int)

X.head()

Loadin ./data/anonLinkedInProfiles.csv: 0it [00:00, ?it/s]

Shape: (1497, 4), does it have NAs:
user            False
descriptions     True
titles           True
class           False
dtype: bool
Resampled Shape: (1400, 5)


0    founder investor using technology to solve the...
1    embrace the way of the success samurai and con...
2    software engineer blackboulder  nyc game devel...
3    i'm a visionary in the world of decentralizati...
4    data scientist strategic and resultsdriven bus...
dtype: object

As with scikit also with tensorflow, though we wish to use APIs that are portable to JS. We have to do some simple word preprocessing:

`String -> Token -> Lemma -> Embeddings`. 

Keras might solve this with their preprocessing.

In [3]:
import nltk

nltk.download('all')

hparams = {
    "batch_size": 64,
    "embedding_dim": 512, # from USE model
    "embedding_trainable": True,
    "learning_rate": 1e-2,
    "dropout_rate": 0.2
}

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\bigworker\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\bigworker\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\bigworker\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\bigworker\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\bigworker\AppData\Roaming\nltk_data.

In [4]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet 
from nltk import pos_tag

WNL = WordNetLemmatizer()
STOP_WORDS = stopwords.words('english')

def clean_sentence(original_text):
  def _get_wordnet_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)
  
  cleaned_text = original_text.strip()
  cleaned_text = original_text.translate(str.maketrans(' ', ' ', string.punctuation))
  cleaned_text = cleaned_text.translate(str.maketrans(' ', ' ', '\n')) # Remove newlines
  cleaned_text = cleaned_text.translate(str.maketrans(' ', ' ', string.digits)) # Remove digits
  cleaned_text = cleaned_text.lower() # Convert to lowercase
  cleaned_text = cleaned_text.split() # Split each sentence using delimiter

  lemmatized_list=[]
  for y in cleaned_text:
    if y in STOP_WORDS:
      continue
    z=WNL.lemmatize(y, _get_wordnet_pos(y))
    lemmatized_list.append(z)

  lemmatized_sentence = ' '.join(lemmatized_list)
  return lemmatized_sentence

X_cleaned = X.progress_apply(lambda text: clean_sentence(text))

  0%|          | 0/1400 [00:00<?, ?it/s]

In [5]:
import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE
import tensorflow_hub as hub

EMBED = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def text_to_dataset(texts,labels):
    return Dataset.from_tensor_slices((EMBED(texts), labels.tolist())).cache().batch(hparams['batch_size'],drop_remainder=True).prefetch(AUTOTUNE)

VAL_SIZE = int(len(X)*0.3)


x_train = X_cleaned[VAL_SIZE:]
y_train = Y[VAL_SIZE:]

x_val = X_cleaned[:VAL_SIZE]
y_val = Y[:VAL_SIZE]

TEST_SIZE = int(len(x_val)*0.3)

x_test = x_val[TEST_SIZE:]
y_test = y_val[TEST_SIZE:]
x_val = x_val[:TEST_SIZE]
y_val = y_val[:TEST_SIZE]

print(f'Sizes for TEST: {TEST_SIZE}, validation: {VAL_SIZE} and train: {len(X) - VAL_SIZE}')

train_ds = text_to_dataset(x_train, y_train)
val_ds = text_to_dataset(x_test, y_test)
test_ds = text_to_dataset(x_val, y_val)

# get top 5 elements of this prefectched slice.
list(test_ds.take(1))[0]

Sizes for TEST: 126, validation: 420 and train: 980


(<tf.Tensor: shape=(64, 512), dtype=float32, numpy=
 array([[-0.03083667, -0.0665146 , -0.0182992 , ...,  0.05561822,
         -0.04941908, -0.01916869],
        [ 0.02800515, -0.0724399 , -0.05752612, ..., -0.00541646,
          0.05067081, -0.07265382],
        [ 0.05944948,  0.02337807,  0.02523481, ...,  0.04628878,
          0.02467602, -0.03657639],
        ...,
        [-0.00581072, -0.07151613,  0.05340949, ..., -0.04404993,
         -0.06780335, -0.00589926],
        [ 0.00980827, -0.05763037, -0.0325496 , ...,  0.06339546,
         -0.06771517, -0.06418051],
        [ 0.01481714, -0.02670703,  0.04458027, ..., -0.01660948,
         -0.04708564, -0.0305433 ]], dtype=float32)>,
 <tf.Tensor: shape=(64,), dtype=int32, numpy=
 array([0, 1, 2, 3, 2, 3, 4, 4, 2, 0, 4, 3, 0, 3, 3, 4, 1, 3, 2, 3, 0, 2,
        1, 2, 2, 3, 2, 2, 0, 3, 1, 3, 3, 1, 0, 1, 1, 2, 3, 3, 1, 1, 0, 3,
        2, 2, 2, 2, 0, 0, 3, 3, 2, 2, 0, 4, 3, 3, 3, 1, 4, 4, 3, 4])>)

Build the model. See: https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub_on_kaggle


In [6]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(512,),batch_size=hparams['batch_size'], name="in_embeddings"),
        layers.Dense(int(hparams['embedding_dim']/4),batch_size=hparams['batch_size'], activation="sigmoid", name="layer1"),
        layers.Dropout(hparams['dropout_rate'],batch_size=hparams['batch_size'], name="drop"),
        layers.BatchNormalization(batch_size=hparams['batch_size']),
        layers.Dense(len(LABEL_DICT),batch_size=hparams['batch_size'], activation="softmax", name="out")
    ]
)

# Load the model and view a summary.
model.compile(
    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.optimizers.Adam(learning_rate=hparams['learning_rate']), 
    metrics = [keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
               keras.metrics.MeanAbsoluteError(name='mean_absolute_error')
            ])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (64, 128)                 65664     
                                                                 
 drop (Dropout)              (64, 128)                 0         
                                                                 
 batch_normalization (BatchN  (64, 128)                512       
 ormalization)                                                   
                                                                 
 out (Dense)                 (64, 5)                   645       
                                                                 
Total params: 66,821
Trainable params: 66,565
Non-trainable params: 256
_________________________________________________________________


Fit model

In [7]:
import tensorflow_addons as tfa

tqdm_callback = tfa.callbacks.TQDMProgressBar()

history = model.fit(train_ds, 
                    epochs=10, 
                    verbose = 10,
                    validation_data = val_ds,
                    callbacks=[tqdm_callback]
                    )

history.history

c:\Users\bigworker\anaconda3\envs\tutorials\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\bigworker\anaconda3\envs\tutorials\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file 

Training:   0%|           0/10 ETA: ?s,  ?epochs/s

Epoch 1/10


0/15           ETA: ?s - 

Epoch 1/10


...

0/15           ETA: ?s - 

Epoch 9/10
Epoch 10/10


0/15           ETA: ?s - 

Epoch 10/10


{'loss': [0.9318115711212158,
  0.30891773104667664,
  0.20785151422023773,
  0.12865020334720612,
  0.12916837632656097,
  0.1035962849855423,
  0.09670994430780411,
  0.07007812708616257,
  0.06051131710410118,
  0.04980221018195152],
 'accuracy': [0.6447916626930237,
  0.8895833492279053,
  0.9270833134651184,
  0.9479166865348816,
  0.953125,
  0.9614583253860474,
  0.9635416865348816,
  0.9750000238418579,
  0.9791666865348816,
  0.9854166507720947],
 'mean_absolute_error': [1.9491666555404663,
  1.9491668939590454,
  1.9491668939590454,
  1.9491667747497559,
  1.9491666555404663,
  1.9491668939590454,
  1.9491668939590454,
  1.9491668939590454,
  1.9491668939590454,
  1.9491668939590454],
 'val_loss': [1.0374902486801147,
  0.8465980291366577,
  0.6793274283409119,
  0.5862947106361389,
  0.5190024971961975,
  0.4587930738925934,
  0.4010266661643982,
  0.3429074287414551,
  0.3244614005088806,
  0.2891292870044708],
 'val_accuracy': [0.375,
  0.56640625,
  0.75,
  0.7578125,
  0

Evaluate and move the model to TensorFlow JS.

In [8]:
from math import floor

print("Evaluating test data")
print(model.evaluate(test_ds, batch_size=hparams['batch_size']))

# Remember all our imputs need to be embedded first!
job_titles = EMBED(["IT Consultant at Sesame Street, lord of Java Code, who likes to learn new stuff and tries some machine learning in my free engineering time."])

print("\nEvaluating new Description")
probas = model.predict(job_titles)[0]
print(LABEL_DICT)
print(probas)

max_proba_idx = np.argmax(probas)
print(f'\nPredicted character: [{list(LABEL_DICT)[max_proba_idx]}] with probability of: [{floor(probas[max_proba_idx]*100.0)}%]')

Evaluating test data
1/1 [==============================] - 0s 14ms/step - loss: 0.2532 - accuracy: 0.8750 - mean_absolute_error: 1.9562
[0.25324320793151855, 0.875, 1.9562499523162842]

Evaluating new Description
1/1 [==============================] - 0s 81ms/step
{'bigbird': 0, 'count': 1, 'grover': 2, 'grouch': 3, 'erniebert': 4}
[0.00157533 0.0025427  0.86415637 0.12862763 0.00309801]

Predicted character: [grover] with probability of: [86%]


In [9]:
MODEL_PATH = 'models/tf'

model.save(MODEL_PATH)

model = keras.models.load_model(MODEL_PATH)
model.predict(job_titles)

INFO:tensorflow:Assets written to: models/tf\assets


INFO:tensorflow:Assets written to: models/tf\assets


1/1 [==============================] - 0s 57ms/step


array([[0.00157533, 0.0025427 , 0.86415637, 0.12862763, 0.00309801]],
      dtype=float32)

In [10]:
import tensorflowjs as tfjs
MODEL_JS_PATH = 'models/tfjs'

tfjs.converters.save_keras_model(model, MODEL_JS_PATH)